# Analysation of latency data
In this notebook we want to analyse all the latency data



### what do i want to analyse?
- distribution of latency
- correlation between input size and latency measures

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import *
import pandas as pd
from pathlib import Path

## 1) Load Dataset

In [ ]:
# Define root results directory
results_dir = Path('../results')

# Find all files containing "stats" and ending with .csv in all subdirectories
stats_files = sorted(results_dir.glob('**/*stats*.csv'))

# Find all files containing "raw" and ending with .csv in all subdirectories
raw_files = sorted(results_dir.glob('**/*raw*.csv'))

print(f"Found {len(stats_files)} stats CSV files and {len(raw_files)} raw CSV files:")

# Load all stats files into a dictionary of dataframes
stats_dfs = {}
for file_path in stats_files:
    try:
        df = pd.read_csv(file_path)
        # Strip whitespace from column names
        df.columns = df.columns.str.strip()
        # Create key: subfolder_name/filename_stem
        relative_path = file_path.relative_to(results_dir)
        key = str(relative_path.parent / relative_path.stem)
        stats_dfs[key] = df
        print(f"✅ {relative_path} -> shape {df.shape}")
    except Exception as e:
        print(f"❌ Error loading {file_path.name}: {e}")

# Load all raw files into a dictionary of dataframes
raw_dfs = {}
for file_path in raw_files:
    try:
        df = pd.read_csv(file_path)
        # Strip whitespace from column names
        df.columns = df.columns.str.strip()
        # Create key: subfolder_name/filename_stem
        relative_path = file_path.relative_to(results_dir)
        key = str(relative_path.parent / relative_path.stem)
        raw_dfs[key] = df
        print(f"✅ {relative_path} -> shape {df.shape}")
    except Exception as e:
        print(f"❌ Error loading {file_path.name}: {e}")

print(f"\n✅ Loaded {len(stats_dfs)} stats dataframes and {len(raw_dfs)} raw dataframes ✅")

In [ ]:
# raw_dfs['fabian\\raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-03T20-41-45']

In [ ]:
# Check the keys in raw_dfs
print("Available keys in raw_dfs:")
for key in raw_dfs.keys():
    print(key)

## 2) Inference Time vs. Accuracy

In [ ]:
# Call the function with the stats_dfs dictionary
# TODO: shorten the labels (maybe just "granite micro" instead of ibm-granite-granite-4-0-h-micro). Would do this with a lookup table. Maybe add directly on data loading step.
# TODO: color the dots based on cloud vs. on device
# TODO: optional: make the size of the dots depending on the model size
scatterplot_inference_vs_accuracy(stats_dfs)

In [ ]:
# plot_time_measure_distributions(raw_dfs['fabian/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-03T20-58-00'], 'Llama Fabian')
# plot_time_measure_distributions(raw_dfs['philip\\raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'], 'Llama Philip')
#plot_time_measure_distributions(raw_dfs['nicolas/lenovo_büro_raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T13-03-58'], 'Llama Nicolas Büro')


In [ ]:
# Prepare the data for plotting
experiment_data = [
    raw_dfs['fabian/raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-03T20-41-45'],
    raw_dfs['philip/raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-04T08-01-13'],
    raw_dfs['nicolas/lenovo_büro_raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-04T12-44-34']
]

labels = ['gemma Fabian', 'gemma Philip', 'gemma Nicolas Büro']

# plot time measure distributions
plot_time_measures_overlaid(experiment_data, labels)

# plot inference vs input character amount
plot_characters_vs_inference_time(experiment_data, labels)

In [ ]:
# Prepare the data for plotting
experiment_data = [
    raw_dfs['fabian/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-03T20-58-00'],
    raw_dfs['philip/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'],
    raw_dfs['nicolas-office/lenovo_büro_raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T13-03-58'],
    raw_dfs['cloud/cloud_raw_experiment_meta-llama-llama-3-2-1b-instruct_always_cloud_once-per-sec_2025-12-04T16-10-54']
]

labels = ['Llama Fabian', 'Llama Philip', 'Llama Nicolas Büro', 'cloud']

# plot time measure distributions
plot_time_measures_overlaid(experiment_data, labels)

# plot inference vs input character amount
plot_characters_vs_inference_time(experiment_data, labels)

In [ ]:
# Prepare the data for plotting
experiment_data_granite = [
    raw_dfs['fabian/raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-03T22-46-10'],
    # Updated key for Fabian
    raw_dfs['philip/raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-04T09-03-29'],
    # Correct key for Philip
    raw_dfs[
        'nicolas/lenovo_büro_raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-04T12-29-10']
    # Correct key for Nicolas
]

labels_granite = ['Granite Fabian', 'Granite Philip', 'Granite Nicolas Büro']

# Plot time measure distributions for Granite experiments
plot_time_measures_overlaid(experiment_data_granite, labels_granite)

# Plot inference vs input character amount for Granite experiments
plot_characters_vs_inference_time(experiment_data_granite, labels_granite)

## n) Analyisis of Latency per Model and Device
This analyiss is used to show the mean and median latency as well as the variance in latency for the on device models of different configurations. Those numbers can be copied to the report.

In [ ]:
# filter raw_df such that only entries with fabian in the path are kept
fabian_raw_dfs = {key: df for key, df in raw_dfs.items() if 'fabian' in key}

In [ ]:
def analyze_latency_statistics_for_one_device(name: str, route: str = 'device') -> pd.DataFrame:
    # filter all the raw dataframes which contain name in their key (e.g. 'fabian' for all datasets from fabian)
    subset_raw_df = {key: df for key, df in raw_dfs.items() if name in key}

    latency_stats = {}
    for key, df in subset_raw_df.items():
        if 'inference_time_ms' not in df.columns:
            continue

        series = df['inference_time_ms'].dropna()
        mean_latency = series.mean()
        median_latency = series.median()
        std_latency = series.std()
        q25 = series.quantile(0.25)
        q75 = series.quantile(0.75)
        iqr_latency = q75 - q25

        mean_accuracy = df['exact_match'].mean() if 'exact_match' in df.columns else None
        device_model = df['device_model'][0]
        cloud_model = df['cloud_model'][0]

        model_name = cloud_model if route == 'cloud' else device_model

        latency_stats[model_name] = {
            'mean_latency_ms': round(mean_latency, 2),
            'median_latency_ms': round(median_latency, 2),
            'std_latency_ms': round(std_latency, 2),
            'iqr_latency_ms': round(iqr_latency, 2),
            'q25_latency_ms': round(q25, 2),
            'q75_latency_ms': round(q75, 2),
            'mean_accuracy': round(mean_accuracy, 2),
            'dataset_name': key
        }

    # convert to a dataframe
    return pd.DataFrame.from_dict(latency_stats, orient='index')


In [ ]:
# print the dataframe
analyze_latency_statistics_for_one_device('fabian')

In [ ]:
analyze_latency_statistics_for_one_device('philip')

In [ ]:
analyze_latency_statistics_for_one_device('nicolas-privat')

In [ ]:
analyze_latency_statistics_for_one_device('nicolas-office')

## 5) Prompt Characters vs. Latency Analysis
This analysis is used to show the correlation between input size (in tokens) and latency for different models and devices.

In [ ]:
# Prepare the data for plotting
experiment_data = [
    [
        raw_dfs['fabian\\raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-03T20-58-00'],
        raw_dfs['philip\\raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'],
        raw_dfs[
            'nicolas-privat\\lenovo_privat_raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-05T08-43-10'],
        raw_dfs[
            'cloud\\cloud_raw_experiment_meta-llama-llama-3-2-1b-instruct_always_cloud_once-per-sec_2025-12-04T16-10-54']
    ],
    [
        raw_dfs['fabian\\raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-03T22-46-10'],
        raw_dfs['philip\\raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-04T09-03-29'],
        raw_dfs[
            'nicolas-privat\\lenovo_privat_raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-05T09-19-31'],
        raw_dfs[
            'cloud\\cloud_raw_experiment_ibm-granite-granite-4-0-h-micro_always_cloud_once-per-sec_2025-12-05T07-07-33']
    ],
    [
        raw_dfs['fabian\\raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-03T20-41-45'],
        raw_dfs['philip\\raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-04T08-01-13'],
        raw_dfs[
            'nicolas-privat\\lenovo_privat_raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-05T08-27-21']
    ],
    [
        raw_dfs['fabian\\raw_experiment_Qwen3-4B-ONNX_always_device_once-per-sec_2025-12-06T10-14-33'],
        raw_dfs['philip\\raw_experiment_Qwen3-4B-ONNX_always_device_once-per-sec_2025-12-04T10-12-42'],
        raw_dfs[
            'nicolas-privat\\lenovo_privat_raw_experiment_Qwen3-4B-ONNX_always_device_once-per-sec_2025-12-05T10-40-39']
    ]

]

labels = [
    ['Asus Zenbook', 'Macbook Air', 'Lenovo', 'Cloud'],
    ['Asus Zenbook', 'Macbook Air', 'Lenovo', 'Cloud'],
    ['Asus Zenbook', 'Macbook Air', 'Lenovo'],
    ['Asus Zenbook', 'Macbook Air', 'Lenovo']
]

model_names = [
    'Llama-3-2-1B-Instruct-ONNX',
    'Granite-4-0-micro-ONNX-web',
    'Gemma-3-270m-it-ONNX',
    'Qwen3-4B-ONNX'
]


# plot inference vs input character amount
plot_characters_vs_inference_time(experiment_data, labels, model_names)

In [ ]:
print("Correlation Analysis between Input Characters and Inference Time for Llama:")
calc_correlation_characters_inference(experiment_data[0], labels[0])

In [ ]:
print("Correlation Analysis between Input Characters and Inference Time for Granite:")
calc_correlation_characters_inference(experiment_data[1], labels[1])

In [ ]:
print("Correlation Analysis between Input Characters and Inference Time for Gemma:")
calc_correlation_characters_inference(experiment_data[2], labels[2])

In [ ]:
print("Correlation Analysis between Input Characters and Inference Time for Qwen:")
calc_correlation_characters_inference(experiment_data[3], labels[3])

## 6) Distribution of Inference Time
This section explores how the inference time is distributed for different models and devices.

In [ ]:
plot_inference_time_distribution(experiment_data, labels, model_names)

## 7) Analysis of Warmup Effects
This section analyzes whether there are warmup effects in the latency data, i.e., whether the latency decreases over time as the model "warms up".

In [ ]:
# select from each experiment the first 50 and last 50 samples
def analyze_warmup_effects(experiment_data, labels):
    for data_list, label_list in zip(experiment_data, labels):
        for df, label in zip(data_list, label_list):
            # Sort by job_start_ts to ensure chronological order
            df_sorted = df.sort_values('job_start_ts').reset_index(drop=True)
            first = df_sorted['inference_time_ms'].iloc[0]
            second = df_sorted['inference_time_ms'].iloc[1]
            diff_first_two = first - second

            # get reduction of time in percentage
            reduction_percentage = (diff_first_two / first) * 100

            print(f"{label}: Mean Inference Time - Difference in first two items: {diff_first_two:.2f} ms, {reduction_percentage:2f}% reduction")

In [ ]:
analyze_warmup_effects(experiment_data, labels)

In [ ]:
plot_warm_up_phase(experiment_data, labels, model_names)

In [ ]:
experiment_data[1][3]

# Queueing Models
Here we try to simulate a queueing model that behaves like the empirically generated data. In a second step we then want to combine on-device and cloud into a multi-server model, to come up with a smart decision policy.

In [ ]:
from utils import *

In [ ]:
lambda_val, mu_val, s_times, ia_times = fit_mm1_model(
    raw_dfs['philip/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'])

In [ ]:
# 1. Get rates from On-Device Experiment
print("--- On-Device Data ---")
lambda_device, mu_device, _, _ = fit_mm1_model(
    raw_dfs['philip/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'])

# 2. Get rates from Cloud Experiment
print("\n--- Cloud Data ---")
cloud_key = 'cloud/cloud_raw_experiment_meta-llama-llama-3-2-1b-instruct_always_cloud_once-per-sec_2025-12-04T16-10-54'
lambda_cloud, mu_cloud, _, _ = fit_mm1_model(raw_dfs[cloud_key])

# 3. Combine them into a 2-Server System
# We assume the total arrival rate is the sum of both experiments (or you can set a target lambda)
total_lambda = lambda_device + lambda_cloud

print("\n--- Combined System (M/M/2) ---")
# We pass the two different service rates
fit_mmc_model(total_lambda, [mu_device, mu_cloud], c=2)

In [ ]:
# --- Usage Example ---
thresholds = range(0, 2000, 1)
tolerance = 50      # how much slower (in ms) do we allow the on device model to be in comparison to the cloud model
cost_device = 0.00
cost_cloud = 0.01

df_cloud_ex = raw_dfs['cloud/cloud_raw_experiment_meta-llama-llama-3-2-1b-instruct_always_cloud_once-per-sec_2025-12-04T16-10-54']
df_device_ex = raw_dfs['philip/raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-04T08-01-13']

_ = run_full_analysis(df_device_ex, df_cloud_ex, thresholds, tolerance, cost_device, cost_cloud)

## Coming up with a routing policy
We want to find out the optimal Threshold T here for our "smart" scheduling policy. On the hardware we tested, mostly the cloud based inference was faster than on-device. We expect that in the future models will get faster and more people will have more access to higher performance hardware in their devices, thats why we lower the inference time for the on-device models by multiplying it with a `on_device_speedup_factor` and performing a linear shift with `on_device_speedup_shift`.

In [ ]:
from utils import *

df_cloud_ex = raw_dfs['cloud/cloud_raw_experiment_meta-llama-llama-3-2-1b-instruct_always_cloud_once-per-sec_2025-12-04T16-10-54'].copy()
df_device_ex = raw_dfs['philip/raw_experiment_Llama-3-2-1B-Instruct-ONNX_always_device_once-per-sec_2025-12-04T08-10-53'].copy()

df_cloud_ex = raw_dfs['cloud/cloud_raw_experiment_ibm-granite-granite-4-0-h-micro_always_cloud_once-per-sec_2025-12-05T07-07-33'].copy()
df_device_ex = raw_dfs['philip/raw_experiment_granite-4-0-micro-ONNX-web_always_device_once-per-sec_2025-12-04T09-03-29'].copy()

df_cloud_ex = raw_dfs['cloud/cloud_raw_experiment_openai-gpt-4o-mini_always_cloud_once-per-sec_2025-12-05T07-18-02'].copy()
df_device_ex = raw_dfs['philip/raw_experiment_gemma-3-270m-it-ONNX_always_device_once-per-sec_2025-12-04T08-01-13'].copy()



on_device_speedup_factor = 1
on_device_speedup_shift = 00 #linear shift factor in ms

# Apply speedup factor to device inference times
print(f"Applying speedup factor {on_device_speedup_factor} and shift -{on_device_speedup_shift}ms...")

# clip(lower=1.0) to prevent negative inference times!
df_device_ex['inference_time_ms'] = (df_device_ex['inference_time_ms'] * on_device_speedup_factor - on_device_speedup_shift).clip(lower=1.0)

# WARNING: We cannot simply recalculate total_latency_ms here.
# The old 'queueing_time_ms' is invalid because a faster device would have had a much smaller queue.
# We must rely on the M/G/1 simulation to estimate the new total latency.

# Recalculate total latency to maintain consistency (Total = Queue + Inference). ... (Don't do this if you want accurate total stats)
# df_device_ex['total_latency_ms'] = df_device_ex['queueing_time_ms'] + df_device_ex['inference_time_ms']


We now extract basic per-server metrics from the (manipulated) experiment data.

In [ ]:
from utils import *

extract_basic_metrics(df_device_ex, "On-Device (Llama)")
extract_basic_metrics(df_cloud_ex, "Cloud (Llama)")

Using the basic per-server metrics we fit a queueing model per server using the measured service time distribution. To find out what queue model we should use. we need to know the distribution of the interarrival times and the distribution of the service times, therefore we plot them.

In [ ]:
plot_inter_arrival_distribution(df_device_ex, "On-Device (Llama)")
plot_inter_arrival_distribution(df_cloud_ex, "Cloud (Llama)")

In [ ]:
plot_service_time_distribution(df_device_ex, "On-Device (Llama)")
plot_service_time_distribution(df_cloud_ex, "Cloud (Llama)")

We can see that the interarrival times are roughly deterministic. This matches our expectations, as we did the experiments with 1 request per second. The service time distribution plots show that the empirical service times are not clearly exponential, with different shapes and variances for device and cloud. Therefore, for analysis, we model our cloud and on-device systems as **G/G/1** queues (General Arrival, General Service).

Since there is no exact closed-form solution for the mean waiting time in a G/G/1 queue, we use **Kingman's Approximation** (see lecture/literature). The expected waiting time $E[T_Q]$ is approximated as:

$$
E[T_Q] \approx \frac{\rho}{1-\rho} \cdot \frac{c_a^2 + c_s^2}{2} \cdot E[S]
$$

Where:
- $\rho = \lambda E[S]$ is the utilization.
- $E[S]$ is the mean service time.
- $c_a$ is the coefficient of variation of inter-arrival times ($c_a = \sigma_a / \mu_a$).
- $c_s$ is the coefficient of variation of service times ($c_s = \sigma_s / \mu_s$).

This formula generalizes the queueing behavior:
1.  **For our Experiment (D/G/1):** Since arrivals are deterministic, $c_a \approx 0$. The waiting time is driven purely by the service variability ($c_s^2$).
2.  **For M/G/1 (Theoretical):** If we assume random Poisson arrivals, $c_a = 1$. In this case, Kingman's formula simplifies back to the Pollaczek–Khinchine formula.

Using this approximation, we can analytically compute the expected mean response time for both our specific experiment ($c_a=0$) and for a hypothetical real-world scenario with random user arrivals ($c_a=1$).

In [ ]:
from utils import *

# --- Run Analysis ---
thresholds = range(0, 2500, 50)
# ca=0.0 models your deterministic experiment (D/G/1)
gg1_results = analyze_routing_gg1(df_device_ex, df_cloud_ex, thresholds, ca=0.0)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(gg1_results['threshold'], gg1_results['avg_latency'], label='G/G/1 Estimated Latency (Kingman)', linewidth=2)
plt.axhline(y=gg1_results.iloc[0]['avg_latency'], color='gray', linestyle='--', label='All Cloud (Baseline)')

finite_vals = gg1_results[gg1_results['avg_latency'] != float('inf')]['avg_latency']
upper_lim = finite_vals.max() * 1.1 if not pd.isna(finite_vals.max()) else 2.0
lower_lim = max(0, finite_vals.min() * 0.9) if not pd.isna(finite_vals.min()) else 0
plt.ylim(lower_lim, upper_lim)

plt.xlabel('Threshold (Characters)')
plt.ylabel('Mean Response Time (s)')
plt.title('G/G/1 Routing Analysis: Kingman\'s Approximation (ca=0.0)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**how to interpret the analyis plot:**
- X-Axis is the Threshold we set for the routing policy. Any prompt with less characters than T is sent to local device, anything longer goes to the cloud.
- Y-Axis is the Mean Response Time $E[R]$ for the combined system (device and cloud). It's the weighted weighted average time of requests served by device and cloud.
- The optimal Threshold is the minimum of the curve

**To validate the analytical G/G/1 setup (Kingman's approximation), we build a discrete‑event simulation.** We run two variations: one with **Deterministic arrivals** (matching our experiment, $c_a=0$) and one with **Poisson arrivals** (matching theoretical random traffic, $c_a=1$). For each arrival, we apply the routing rule (if size $\le T$ route to device, otherwise to cloud) and simulate the two FCFS queues using empirically sampled service times. For each threshold $T$, we estimate the average response time over many simulated jobs (`num_jobs`) and compare it to the analytical curves. This verifies that the approximation holds for both our specific experimental conditions and general random workloads.

In [ ]:
from utils import *

# --- Run Simulation ---
# Use fewer thresholds or jobs if it's too slow
sim_thresholds = range(0, 2500, 50)

# Calculate Lambda once
lambda_sim = calculate_system_arrival_rate(df_device_ex, df_cloud_ex)

# Run Simulation with Deterministic Arrivals (ca=0.0)
sim_results_det = simulate_routing_validation(df_device_ex, df_cloud_ex, sim_thresholds, lambda_sim, num_jobs=20000, ca=0.0)

# Run Simulation with Poisson Arrivals (ca=1.0)
sim_results_poisson = simulate_routing_validation(df_device_ex, df_cloud_ex, sim_thresholds, lambda_sim, num_jobs=20000, ca=1.0)

# --- Plot Comparison ---
plt.figure(figsize=(10, 6))
# Note: Using gg1_results from the previous cell (Analytical Model)
plt.plot(gg1_results['threshold'], gg1_results['avg_latency'], label='Analytical (G/G/1 Kingman)', linewidth=2, color='blue')
plt.plot(sim_results_poisson['threshold'], sim_results_poisson['sim_latency'], 'o--', label='Sim (Poisson, ca=1)', color='orange', alpha=0.5)
plt.plot(sim_results_det['threshold'], sim_results_det['sim_latency'], 'x--', label='Sim (Deterministic, ca=0)', color='green')

# Baseline
plt.axhline(y=gg1_results.iloc[0]['avg_latency'], color='gray', linestyle='--', label='All Cloud (Baseline)')

# Dynamic Limits
finite_vals = gg1_results[gg1_results['avg_latency'] != float('inf')]['avg_latency']
upper_lim = finite_vals.max() * 1.1 if not pd.isna(finite_vals.max()) else 2.0
lower_lim = max(0, finite_vals.min() * 0.9) if not pd.isna(finite_vals.min()) else 0
plt.ylim(lower_lim, upper_lim)

plt.xlabel('Threshold (Characters)')
plt.ylabel('Mean Response Time (s)')
plt.title('Validation: G/G/1 Analytical Model vs. Discrete Event Simulation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

The discrete-event simulation shows slightly lower latency than the Kingman approximation. This is expected, as Kingman's formula is an upper-bound approximation that tends to overestimate waiting times for deterministic arrivals at non-saturated utilization levels.


Now we want to look at a fully simulated version. Here we don't sample arrival- and inference time from our experiments, but we generate them artificially. Earlier we identified a linear relationship between input size and inference time, we will use this to generate accurate distributions to sample pairs of input sizes and inference times.

In [ ]:
# Get parameters for Device
slope_dev, int_dev = estimate_linear_relationship(df_device_ex, "On-Device (Llama)")

# Get parameters for Cloud
slope_cloud, int_cloud = estimate_linear_relationship(df_cloud_ex, "Cloud (Llama)")

# Get parameters for Input Character Distribution
char_mean = df_device_ex['number_of_characters'].mean()
char_std = df_device_ex['number_of_characters'].std()

print(f"\nSimulation Params -> Char: ({char_mean:.1f}, {char_std:.1f})")

We can now take the slope and intercept of the cloud and on-device models and use them to generate sample jobs for our simulation. We will generate Gaussian distributed input sizes and calculate the corresponding inference times using the linear models.

In [ ]:
from utils import *

# --- Run Simulation ---
# Use fewer thresholds or jobs if it's too slow
sim_thresholds = range(0, 2500, 50)

# Calculate Lambda once
lambda_sim = calculate_system_arrival_rate(df_device_ex, df_cloud_ex)
print(lambda_sim)

# Run Simulation with Deterministic Arrivals (ca=0.0)
sim_results_det = simulate_routing_synthetic(thresholds, lambda_sim, num_jobs=10000, ca=0.0, char_params=(char_mean, char_std), dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud))

# Run Simulation with Poisson Arrivals (ca=1.0)
sim_results_poisson = simulate_routing_synthetic(thresholds, lambda_sim, num_jobs=10000, ca=1.0, char_params=(char_mean, char_std), dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud))

# --- Plot Comparison ---
plt.figure(figsize=(10, 6))
# Note: Using gg1_results from the previous cell (Analytical Model)
plt.plot(gg1_results['threshold'], gg1_results['avg_latency'], label='Analytical (G/G/1 Kingman)', linewidth=2, color='blue')
#plt.plot(sim_results_poisson['threshold'], sim_results_poisson['sim_latency'], 'o--', label='Sim (Poisson, ca=1)', color='orange', alpha=0.5)
plt.plot(sim_results_det['threshold'], sim_results_det['sim_latency'], 'x--', label='Sim (Deterministic, ca=0)', color='green')

# Baseline
plt.axhline(y=gg1_results.iloc[0]['avg_latency'], color='gray', linestyle='--', label='All Cloud (Baseline)')
plt.axhline(y=gg1_results.iloc[-1]['avg_latency'], color='red', linestyle=':', label='All Device (Baseline)')

# Dynamic Limits
finite_vals = gg1_results[gg1_results['avg_latency'] != float('inf')]['avg_latency']
upper_lim = finite_vals.max() * 1.1 if not pd.isna(finite_vals.max()) else 2.0
lower_lim = max(0, finite_vals.min() * 0.9) if not pd.isna(finite_vals.min()) else 0
plt.ylim(lower_lim, upper_lim)

plt.xlabel('Threshold (Characters)')
plt.ylabel('Mean Response Time (s)')
plt.title('Validation: G/G/1 Analytical Model vs. Discrete Event Simulation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

We have successfully created a model where we can specify the relationship between input size and inference time for two G/G/1 systems, connected by a threshold-based scheduling policy. This allows us to simulate and optimize the system under various theoretical workloads (deterministic or poisson arrival) and hardware configurations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a range of arrival rates to test (e.g., 0.5 to 3.0 requests per second)
#test_lambdas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0, 1.1, 1.2, 1.3, 1.4, 2]
test_lambdas = [0.1,0.2,0.3]

plt.figure(figsize=(12, 7))
colors = plt.cm.viridis(np.linspace(0, 1, len(test_lambdas)))

print("Running simulations for different arrival rates...")

all_finite_latencies = []

for i, lam in enumerate(test_lambdas):
    print(f"  -> Simulating λ = {lam:.1f} req/s")

    # Run synthetic simulation (using Poisson arrivals ca=1.0 for realistic random traffic)
    sim_res = simulate_routing_synthetic(
        thresholds,
        lam,
        num_jobs=10000,
        ca=1.0, # 1.0 is poisson, 0.0 is deterministc
        char_params=(char_mean, char_std),
        dev_model=(slope_dev, int_dev),
        cloud_model=(slope_cloud, int_cloud)
    )

    # Plot the curve
    plt.plot(sim_res['threshold'], sim_res['sim_latency'],
             label=f'λ = {lam:.1f}', color=colors[i], linewidth=2)

    # Collect finite values for auto-scaling
    finite_vals = sim_res[sim_res['sim_latency'] != float('inf')]['sim_latency']
    all_finite_latencies.extend(finite_vals.dropna().tolist())

plt.xlabel('Threshold (Characters)')
plt.ylabel('Mean Response Time (s)')
plt.title('Impact of Arrival Rate (λ) on Optimal Routing Threshold')
plt.legend(title="Arrival Rate (req/s)")
plt.grid(True, alpha=0.3)

# Set reasonable Y-limits to ignore unstable/infinite queues
if all_finite_latencies:
    upper_lim = max(all_finite_latencies) * 1.1
    lower_lim = max(0, min(all_finite_latencies) * 0.9)
    plt.ylim(lower_lim, upper_lim)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a range of arrival rates to test (e.g., 0.5 to 3.0 requests per second)
#test_lambdas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0, 1.1, 1.2, 1.3, 1.4, 2]
test_lambdas = [0.1,0.2,0.3]

plt.figure(figsize=(12, 7))
colors = plt.cm.viridis(np.linspace(0, 1, len(test_lambdas)))

print("Running simulations for different arrival rates...")

all_finite_latencies = []

for i, lam in enumerate(test_lambdas):
    print(f"  -> Simulating λ = {lam:.1f} req/s")

    # Run synthetic simulation (using Poisson arrivals ca=1.0 for realistic random traffic)
    sim_res = simulate_routing_synthetic(
        thresholds,
        lam,
        num_jobs=10000,
        ca=0.0, # 1.0 is poisson, 0.0 is deterministc
        char_params=(char_mean, char_std),
        dev_model=(slope_dev, int_dev),
        cloud_model=(slope_cloud, int_cloud)
    )

    # Plot the curve
    plt.plot(sim_res['threshold'], sim_res['sim_latency'],
             label=f'λ = {lam:.1f}', color=colors[i], linewidth=2)

    # Collect finite values for auto-scaling
    finite_vals = sim_res[sim_res['sim_latency'] != float('inf')]['sim_latency']
    all_finite_latencies.extend(finite_vals.dropna().tolist())

plt.xlabel('Threshold (Characters)')
plt.ylabel('Mean Response Time (s)')
plt.title('Impact of Arrival Rate (λ) on Optimal Routing Threshold')
plt.legend(title="Arrival Rate (req/s)")
plt.grid(True, alpha=0.3)

# Set reasonable Y-limits to ignore unstable/infinite queues
if all_finite_latencies:
    upper_lim = max(all_finite_latencies) * 1.1
    lower_lim = max(0, min(all_finite_latencies) * 0.9)
    plt.ylim(lower_lim, upper_lim)

plt.show()

### Deriving an input size based policy to send to device or cloud

In [ ]:
import pandas as pd
import time
import random

# 1. Load the dataset
# Assuming the notebook is in 'analyse/' and the dataset is in 'dataset/' relative to the project root
dataset_path = Path('../dataset/boolq_validation.csv')

if not dataset_path.exists():
    print(f"⚠️ File not found at {dataset_path}. Please check the path.")
else:
    df_requests = pd.read_csv(dataset_path)
    print(f"✅ Loaded {len(df_requests)} requests from {dataset_path.name}")

    # 2. Initialize Estimator
    # Using a 5-second window to react relatively quickly to changes in our simulation loop
    estimator = TrafficRateEstimator(window_size_seconds=5)

    print("\n--- Streaming Requests from Dataset ---")

    # 3. Iterate through requests
    for i, row in df_requests.head(20).iterrows():

        # Calculate input size (Question + Passage)
        input_text = str(row['question']) + " " + str(row['passage'])
        input_len = len(input_text)

        # Register the arrival of this request
        estimator.register_request()

        # Get current load estimate
        current_lambda = estimator.get_current_lambda()

        # Make routing decision
        decision, opt_threshold, _ = recommend_routing_decision(
            input_len,
            current_lambda,
            (char_mean, char_std),
            (slope_dev, int_dev),
            (slope_cloud, int_cloud)
        )

        print(f"Req {i+1:02d}: Size={input_len:4d} chars | Est. λ={current_lambda:.2f} | Threshold={opt_threshold} -> Decision: {decision}")

        # Simulate variable traffic load with BURSTS
        # Every 20 requests, switch between "High Traffic" (burst) and "Low Traffic" (lull)
        if (i // 20) % 2 == 0:
            # High Traffic Mode: Fast arrivals (0.01s - 0.2s) -> High Lambda
            time.sleep(random.uniform(0.01, 0.1))
        else:
            # Low Traffic Mode: Slow arrivals (0.5s - 1.5s) -> Low Lambda
            time.sleep(random.uniform(0.5, 1.5))

The experiment above shows something that is not fulfilled in our policy: the scheduler does not keep track of how many elements are present in which queue. to avoid overfilling one queue while the other system is idling, we should keep track of how many requests are in the queues of cloud and on device

### including states to keep track of queue lengths
the scheduler is effectively asking: "Including the current backlog, which server will finish this specific job faster?" This is the core principle of the "Join the Shortest Expected Queue" (JSEQ) policy

In [ ]:
# join shortest queue size
from utils import *

# 1. Load the dataset
df_requests = pd.read_csv(dataset_path)
print(f"✅ Loaded {len(df_requests)} requests from {dataset_path.name}")

# 2. Initialize Stateful Scheduler with our device/cloud performance models
# Ensure slope_dev, int_dev, etc. have been defined in a previous cell
scheduler = StatefulScheduler(dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud))

print("\n--- Streaming Requests with Stateful JSEQ Scheduler (Advanced Simulation) ---")

# --- Advanced Simulation Setup ---
# We get rid of time.sleep() and manage time manually for a more accurate simulation.
sim_time = 0.0  # This is our simulation's internal clock
total_latency = 0.0
num_requests_processed = 0

# 3. Iterate through requests
for i, row in df_requests.head(500).iterrows():

    # Determine next arrival time based on traffic mode
    if (i // 20) % 2 == 0: # Burst mode
        arrival_delay = random.uniform(0.005, 0.15)
    else: # Lull mode
        arrival_delay = random.uniform(0.8, 1.5)

    # Advance simulation time to the next arrival
    sim_time += arrival_delay

    # Calculate input size
    input_text = str(row['question']) + " " + str(row['passage'])
    input_len = len(input_text)

    # Make a state-aware routing decision using the simulation's clock
    decision, start_time, finish_time = scheduler.decide_at_time(input_len, arrival_time=sim_time)

    # Calculate and record stats for this request
    queue_time = start_time - sim_time
    service_time = finish_time - start_time
    response_time = finish_time - sim_time # This is queue_time + service_time

    total_latency += response_time
    num_requests_processed += 1

    # Convert to milliseconds for more readable output
    queue_time_ms = queue_time * 1000
    service_time_ms = service_time * 1000

    # Use '<7' to left-align the decision string in a 7-character space
    print(f"Req {i+1:03d} (t={sim_time:6.2f}s): Size={input_len:4d} -> {decision:<7} | Queue: {queue_time_ms:6.1f}ms, Inference: {service_time_ms:6.1f}ms")

print(f"\n--- Simulation Complete ---")
print(f"Average Response Time over {num_requests_processed} requests: {total_latency / num_requests_processed:.4f}s")

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from utils import *

# --- 1. SETUP: Ensure all parameters from previous cells are available ---
# dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud)
# char_params=(char_mean, char_std)
# dataset_path

# --- 2. GENERATE A CONSISTENT REQUEST STREAM ---
# Create a fixed list of requests and arrival times to ensure every policy is tested on the exact same workload.
print("--- Generating a consistent once-per-second request stream for simulation ---")
df_requests = pd.read_csv(dataset_path)
num_sim_requests = 500
request_stream = []

# The loop index 'i' will serve as the arrival time in seconds (0, 1, 2, ...)
for i, row in df_requests.head(num_sim_requests).iterrows():
    arrival_time = float(i)

    input_text = str(row['question']) + " " + str(row['passage'])
    input_len = len(input_text)

    request_stream.append({'id': i, 'arrival_time': arrival_time, 'size': input_len})

# --- 3. DEFINE A GENERIC SIMULATION RUNNER ---
def run_policy_simulation(scheduler, requests):
    """Runs a simulation for a given scheduler and request stream."""
    total_latency = 0.0
    for req in requests:
        _, _, finish_time = scheduler.decide_at_time(req['size'], req['arrival_time'])
        response_time = finish_time - req['arrival_time']
        total_latency += response_time

    return total_latency / len(requests) if requests else 0.0

# --- 4. PRE-CALCULATE OPTIMAL THRESHOLD FOR THE STATELESS POLICY ---
# Calculate the average lambda of our generated stream to find a reasonable fixed threshold.
avg_inter_arrival_time = request_stream[-1]['arrival_time'] / len(request_stream)
avg_lambda = 1.0 / avg_inter_arrival_time
print(f"\nAverage arrival rate (λ) for the stream: {avg_lambda:.2f} req/s")

# Find the optimal threshold for this average lambda
thresholds = range(0, 2500, 1)
sim_results = simulate_routing_synthetic(
    thresholds, avg_lambda, num_jobs=5000, ca=1.0,
    char_params=(char_mean, char_std),
    dev_model=(slope_dev, int_dev),
    cloud_model=(slope_cloud, int_cloud)
)
# Find the threshold that gives the minimum latency
optimal_T_stateless = sim_results.loc[sim_results['sim_latency'].idxmin()]['threshold']
print(f"Optimal fixed threshold (T) for stateless policy: {optimal_T_stateless} characters")

# 7. Add Cost Info Text
info_text = (f"Cost Settings:\n"
             f"Device: ${cost_device_per_req:.2f}/req\n"
             f"Cloud:  ${cost_cloud_per_req:.2f}/req")
ax1.text(0.8, 0.95, info_text, transform=ax1.transAxes,
         fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# --- 5. INITIALIZE ALL SCHEDULERS ---
# We use the StatefulScheduler class for all, but with routing logic overridden for simpler policies.
# This ensures they all share the same underlying queue management mechanics.

# a) Stateful JSEQ Scheduler (no forced policy)
scheduler_jseq = StatefulScheduler(dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud))

# b) Always-Cloud Scheduler (forced policy)
scheduler_cloud = StatefulScheduler(dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud), force_policy='cloud')

# c) Always-Device Scheduler (forced policy)
scheduler_device = StatefulScheduler(dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud), force_policy='device')

# d) Stateless Threshold Scheduler
# For this one, we still need to override the main decision logic, as it's a unique policy.
scheduler_stateless = StatefulScheduler(dev_model=(slope_dev, int_dev), cloud_model=(slope_cloud, int_cloud))
# Override the main decision method to implement the fixed threshold logic
scheduler_stateless.decide_at_time = lambda size, time: scheduler_stateless.route_to_device(size, time) if size <= optimal_T_stateless else scheduler_stateless.route_to_cloud(size, time)


# --- 6. RUN SIMULATIONS AND GATHER RESULTS ---
print("\n--- Running simulations for all policies ---")
results = {}

results['Always Device'] = run_policy_simulation(scheduler_device, request_stream)
print(f"Finished: Always Device")

results['Always Cloud'] = run_policy_simulation(scheduler_cloud, request_stream)
print(f"Finished: Always Cloud")

results['Stateless Threshold'] = run_policy_simulation(scheduler_stateless, request_stream)
print(f"Finished: Stateless Threshold (T={optimal_T_stateless})")

results['Stateful JSEQ'] = run_policy_simulation(scheduler_jseq, request_stream)
print(f"Finished: Stateful JSEQ")

In [ ]:
# --- 7. ADD EMPIRICAL INFERENCE TIMES ---
# Calculate the average inference times from the original dataframes
avg_device_inference_ms = df_device_ex['total_latency_ms'].mean()
avg_cloud_inference_ms = df_cloud_ex['total_latency_ms'].mean()

# Add them to the results dictionary, converting from ms to seconds
results['Measured Device Total Latency'] = avg_device_inference_ms / 1000.0
results['Measured Cloud Total Latency'] = avg_cloud_inference_ms / 1000.0


# --- 8. PLOT AND DISPLAY RESULTS ---
print("\n--- Simulation Results ---")
for policy, latency in results.items():
    print(f"{policy:<30}: {latency:.4f}s average response time")

# Define the categories for the x-axis
categories = ['Always Device', 'Always Cloud', 'Stateless Threshold', 'Stateful JSEQ']
x = np.arange(len(categories))  # the label locations

# Data for the simulated response time bars
sim_latencies = {
    'Always Device': results['Always Device'],
    'Always Cloud': results['Always Cloud'],
    'Stateless Threshold': results['Stateless Threshold'],
    'Stateful JSEQ': results['Stateful JSEQ']
}
# Data for the measured total latency bars
measured_latencies = {
    'Always Device': results['Measured Device Total Latency'],
    'Always Cloud': results['Measured Cloud Total Latency']
}

bar_width = 0.35  # Width of each bar

fig, ax = plt.subplots(figsize=(12, 8))

# --- Plotting the bars ---
ax.bar(x[0] - bar_width/2, sim_latencies['Always Device'], bar_width,
       label='Simulated Response Time always Device', color='#ff9999')
ax.bar(x[0] + bar_width/2, measured_latencies['Always Device'], bar_width,
       label='Measured total Latency always Device', color='#e60000')
ax.bar(x[1] - bar_width/2, sim_latencies['Always Cloud'], bar_width,
       label='Simulated Response Time always Device', color='#66b3ff')
ax.bar(x[1] + bar_width/2, measured_latencies['Always Cloud'], bar_width,
       label='Measured total Latency always Cloud', color='#0059b3')
ax.bar(x[2], sim_latencies['Stateless Threshold'], bar_width,
       label='Simulated Response Time Threshold Scheduler', color='#99ff99')
ax.bar(x[3], sim_latencies['Stateful JSEQ'], bar_width,
       label='Simulated Response Time Stateful JSEQ', color='#ffcc99')


# Add some text for labels, title and axes ticks
ax.set_ylabel('Average Time (s)')
ax.set_title('Comparison of Scheduling Policies: Simulated vs. Measured Latency')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Function to attach a text label above each bar
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}s',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

# Label all bars plotted
autolabel(ax.patches)

fig.tight_layout()
plt.show()

In [ ]:
df_jseq_ex = raw_dfs['philip/raw_experiment_gemma-3-270m-it-ONNX_openai-gpt-4o-mini_jseq_once-per-sec_2025-12-08T17-12-38'].copy()

# --- 7. ADD EMPIRICAL INFERENCE TIMES ---
# Calculate the average inference times from the original dataframes
avg_device_inference_ms = df_device_ex['total_latency_ms'].mean()
avg_cloud_inference_ms = df_cloud_ex['total_latency_ms'].mean()
avg_jseq_inference_ms = df_jseq_ex['total_latency_ms'].mean()

# Add them to the results dictionary, converting from ms to seconds
results['Measured Device Total Latency'] = avg_device_inference_ms / 1000.0
results['Measured Cloud Total Latency'] = avg_cloud_inference_ms / 1000.0
results['Measured JSEQ Total Latency'] = avg_jseq_inference_ms / 1000.0


# --- 8. PLOT AND DISPLAY RESULTS ---
print("\n--- Simulation Results ---")
for policy, latency in results.items():
    print(f"{policy:<30}: {latency:.4f}s average response time")

# Define the categories for the x-axis
categories = ['Always Device', 'Always Cloud', 'Stateless Threshold', 'Stateful JSEQ']
x = np.arange(len(categories))  # the label locations

# Data for the simulated response time bars
sim_latencies = {
    'Always Device': results['Always Device'],
    'Always Cloud': results['Always Cloud'],
    'Stateless Threshold': results['Stateless Threshold'],
    'Stateful JSEQ': results['Stateful JSEQ']
}
# Data for the measured total latency bars
measured_latencies = {
    'Always Device': results['Measured Device Total Latency'],
    'Always Cloud': results['Measured Cloud Total Latency'],
    'Stateful JSEQ': results['Measured JSEQ Total Latency']
}

bar_width = 0.35  # Width of each bar

fig, ax = plt.subplots(figsize=(12, 8))

# --- Plotting the bars ---
ax.bar(x[0] - bar_width/2, sim_latencies['Always Device'], bar_width,
       label='Simulated Response Time always Device', color='#ff9999')
ax.bar(x[0] + bar_width/2, measured_latencies['Always Device'], bar_width,
       label='Measured total Latency always Device', color='#e60000')
ax.bar(x[1] - bar_width/2, sim_latencies['Always Cloud'], bar_width,
       label='Simulated Response Time always Device', color='#66b3ff')
ax.bar(x[1] + bar_width/2, measured_latencies['Always Cloud'], bar_width,
       label='Measured total Latency always Cloud', color='#0059b3')
ax.bar(x[2], sim_latencies['Stateless Threshold'], bar_width,
       label='Simulated Response Time Threshold Scheduler', color='#99ff99')
ax.bar(x[3] - bar_width/2, sim_latencies['Stateful JSEQ'], bar_width,
       label='Simulated Response Time Stateful JSEQ', color='#ffcc99')
ax.bar(x[3] + bar_width/2, measured_latencies['Stateful JSEQ'], bar_width,
       label='Measured total Latency stateful JSEQ', color="#e67b00")


# Add some text for labels, title and axes ticks
ax.set_ylabel('Average Time (s)')
ax.set_title('Comparison of Scheduling Policies: Simulated vs. Measured Latency')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Function to attach a text label above each bar
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}s',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

# Label all bars plotted
autolabel(ax.patches)

fig.tight_layout()
plt.show()